In [1]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [3]:
filename = 'Stopwords.txt'
with open (filename, "r") as myfile:
    data = myfile.readlines()
Stopwords = [word[0:-1] for word in data if len(word[0:-1])]
print(Stopwords)

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'cc', 'couldn', "couldn't", 'd', 'delivered-to', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'errors-to', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'in-reply-to', 'into', 'is', 'isn', "isn't", 'it', 'its', "it's", 'itself', 'just', 'list-archive', 'list-help', 'list-post', 'list-subscribe', 'list-unsubscribe', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'mime-version', 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'o

In [4]:
df_2018 = getDataframe(2018)
df_2018 = addTokenizedColumnofTitle(df_2018,Stopwords)

In [5]:
priorProbabilities = getPriorProbabilities(df_2018)
print(priorProbabilities)

{'ask_hn': 0.04516455147762353, 'poll': 9.026411279403534e-05, 'show_hn': 0.03691802213276046, 'story': 0.9178271622768219}


In [6]:
AllClasses = list(np.unique(df_2018['Post Type']))
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [7]:
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)
totalNumberOfWords = getTotalWordCount(words_2018)
words_2018 = obtainDataframeWithClassProbabilities(words_2018, AllClasses, delta, appendClassPrefix)

In [8]:
writeModel(words_2018,'stopword-model.txt',AllClasses,appendClassPrefix)

In [9]:
words_2018 = renameModelRows(words_2018, AllClasses, appendClassPrefix)

In [10]:
model = [words_2018, priorProbabilities]

In [11]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData)
testData = testData.reset_index()

In [ ]:
testData = generateCondClassProb(testData, model)
testData.head()

In [ ]:
testData['predicted'] = generatePrediction(testData,AllClasses)
testData.head()   

In [ ]:
testResults = comparePredictions(testData,AllClasses)
testResults.head()

In [ ]:
writeDataframe(testResults,'stopword-result.txt')

In [ ]:
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)

In [ ]:
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
print(df_confusion)

In [ ]:
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)